In [1]:
from openpyxl import load_workbook
import sys
import csv
import pandas as pd

In [2]:
def readCargoInfo(ws):
    '''
    This function is to read information about item need to ship today
    and store in a dictionary itemWeightNum
    {item serial number: [item weight, number of this item need to ship today]}
    '''
    i=2
    itemWeightNum = {}
    while ws.cell(row=i,column=1).value!=None:
        tmp = []
        if ws.cell(row=i,column=3).value!=None:
            tmp = [ws.cell(row=i,column=2).value,ws.cell(row=i,column=3).value]
            #check if the value is reasonable
            if type(tmp[0])!=int and type(tmp[0])!=float:
                with open('error.txt','a') as f:
                    f.write('Please input numeric cargo weight\n')
                    return 0
                    #sys.exit()
            if tmp[0]<0:
                with open('error.txt','a') as f:
                    f.write('Please input non-negative cargo weight\n')
                    return 0
                    #sys.exit()
            if type(tmp[1])!=int or tmp[1]<0:
                with open('error.txt','a') as f:
                    f.write('Please input int positive cargo number\n')
                    return 0
                    #sys.exit()
            itemWeightNum[ws.cell(row=i,column=1).value] = tmp
        i = i + 1
    return itemWeightNum

In [3]:
def readTruckInfo(ws):
    '''
    This function is to read information about trucks that are available today
    and store in a dictionary truckCapacity
    {truck seial number(start by 1, from up to down): truck capacity}
    '''
    i=2
    truckCapacity = {}
    while ws.cell(row=i,column=6).value!=None:
        this = ws.cell(row=i,column=6).value
        if type(this)!=int and type(this)!=float:
            with open('error.txt','a') as f:
                f.write('Please input numeric truck capacity\n')
                return 0
                #sys.exit()
        if this<=0:
            with open('error.txt','a') as f:
                f.write('Please input positive truck capacity\n')
                return 0
        truckCapacity[i-1] = this
        i = i + 1
    return truckCapacity

In [4]:
###our code's aim to to put same item together and use less trucks.

In [5]:
def findMinDiff(itemWeightNum, truckCapacity):
    '''
    This funciton returns the minimun difference between remain total weight and remain truck capacity
    and a list contains the corresponding item serial number and truck seiral number
    Remind: difference -10 is greater than +5, for abs(-10)>abs(+5)
    '''
    #remainWeight is a dictionary, key is the serial number of the cargo that hasnt loaded in the truck
    #value is the remain total weight that still waits to be loaded
    remainWeight = {}
    for k, v in itemWeightNum.items():
        remainWeight[k] = v[0]*v[1]
    #Diff is dictionary, key is the serial number of the cargo
    #value is list, capacacity of eack truck - this item's remain total weight
    Diff = {}
    flag = False #flag means if the minDiff is greater than zero
    #G represents greater than 0, L represents less than 0
    
    minDiffG = float('inf'); minDiffGitemtruck = [-1,-1];
    minDiffL = float('inf'); minDiffLitemtruck = [-1,-1];
    for k,v in remainWeight.items():
        tmp = []
        for cap in truckCapacity.values():
            tmp.append(cap-v)
            if cap-v>=0 and abs(cap-v)<minDiffG:
                minDiffG = cap-v
                i = list(truckCapacity.values()).index(cap)+1
                minDiffGitemtruck = [k,i]
                flag = True
            if cap-v<0 and abs(cap-v)<minDiffL:
                minDiffL = cap-v
                i = list(truckCapacity.values()).index(cap)+1
                minDiffLitemtruck = [k,i]
        Diff[k] = tmp
    if flag:
        minDiff = minDiffG
        minDiffitemtruck = minDiffGitemtruck
    else:
        minDiff = minDiffL
        minDiffitemtruck = minDiffLitemtruck
    return minDiff, minDiffitemtruck

In [6]:
def actionGreaterThanZero(inputLog, itemWeightNum, truckCapacity,minDiffitemtruck,printAction=False):
    '''
    When the min Difference greater than or equal to 0,
    do a corresponding action: put all remain of one item in one truck,
    modify itemWeightNum and truckCapacity(Delete truck that full, delete item all loaded) 
    and add the action in inputLog
    '''
    #put which item  into which truck, and the number of the item
    #print(minDiffitemtruck)
    whichItem = minDiffitemtruck[0]
    whichTruck=minDiffitemtruck[1]
    howmanyItem=itemWeightNum[whichItem][1]
    action = [whichItem, whichTruck, howmanyItem]
    if printAction:
        print('Action: Move item',whichItem,'into truck',whichTruck,', moving number:',howmanyItem)
    inputLog.append(action)
    itemWeightNum[whichItem][1]=itemWeightNum[whichItem][1]-howmanyItem
    truckCapacity[whichTruck]=truckCapacity[whichTruck]-howmanyItem*itemWeightNum[whichItem][0]
    itemWeightNum.pop(whichItem)
    if truckCapacity[whichTruck] == 0:
        truckCapacity.pop(whichTruck)
    return itemWeightNum, truckCapacity, inputLog

In [7]:
def actionLessThanZero(inputLog, itemWeightNum, truckCapacity,minDiffitemtruck, printAction=False):
    '''When the min Difference less than 0,
    do a corresponding action: put as much as possible of one item in one truck,
    modify itemWeightNum and truckCapacity((Delete truck that full, Delete item cannot fit in)
    and add the action in inputLog'''
    cantLoadCargo = {}
    whichItem = minDiffitemtruck[0];whichTruck=minDiffitemtruck[1]
    itemSingleWeight=itemWeightNum[whichItem][0]
    howmanyItem=truckCapacity[whichTruck]//itemSingleWeight
    #If cannot put any of this item in it, print out the message
    if howmanyItem==0:
        if printAction:
            print('Can not put remaining',itemWeightNum[whichItem][1],'of',whichItem,'in any truck!')
        cantLoadCargo[whichItem] = itemWeightNum[whichItem][1]
        itemWeightNum.pop(whichItem)
        return itemWeightNum, truckCapacity, inputLog, cantLoadCargo
    action = [whichItem, whichTruck, howmanyItem]
    if printAction:
        print('Action: Move item',whichItem,'into truck',whichTruck,', moving number:',howmanyItem)
    inputLog.append(action)
    itemWeightNum[whichItem][1]=itemWeightNum[whichItem][1]-howmanyItem
    truckCapacity[whichTruck]=truckCapacity[whichTruck]-howmanyItem*itemWeightNum[whichItem][0]
    if itemWeightNum[whichItem][1]==0:
        itemWeightNum.pop(whichItem)
        remainCargo.pop(whichItem)
    if truckCapacity[whichTruck] == 0:
        truckCapacity.pop(whichTruck)
    return itemWeightNum, truckCapacity, inputLog, cantLoadCargo

In [8]:
def loadLoop(ws):
    '''
    Main loop of the program
    '''
    itemWeightNum=readCargoInfo(ws)
    truckCapacity = readTruckInfo(ws)
    inputLog = []
    cantLoadCargo = {}
    while itemWeightNum!={}:
        minDiff, minDiffitemtruck = findMinDiff(itemWeightNum, truckCapacity)
        if minDiff>=0:
            itemWeightNum, truckCapacity, inputLog=actionGreaterThanZero(inputLog, itemWeightNum, truckCapacity, minDiffitemtruck=minDiffitemtruck)
        if minDiff<0:
            itemWeightNum, truckCapacity, inputLog, cantLoadCargo=actionLessThanZero(inputLog, itemWeightNum, truckCapacity, minDiffitemtruck=minDiffitemtruck)
    return inputLog, itemWeightNum, truckCapacity, cantLoadCargo

In [9]:
def writeInFile(inputLog, cantLoadCargo, truckCapacity):
    '''
    Write inputLog, itemWeightNum, truckCapacity into csv file
    '''
    with open('inputLog.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Item','Truck','MoveNum'])
        writer.writerows(inputLog)
    if cantLoadCargo!={}:
        df1 = pd.DataFrame({'RemainItem':cantLoadCargo.keys(),'RemainNumber':cantLoadCargo.values()})
        df1.to_csv('RemainItemNum.csv',index=False)
    if truckCapacity!={}:
        df2 = pd.DataFrame({'RemainTruck':truckCapacity.keys(),'RemainCapacity':truckCapacity.values()})
        df2.to_csv('RemainTruckCapacity.csv',index=False)

In [10]:
#data_dir = input('Please input your data dir:') #project.xlsx
def main():
    data_dir = input('Please input your data path')#"project.xlsx"
    #data_dir = 'aaa.xlsx'
    #sheetName = 'DATA'
    sheetName = input('Please input your sheetname')
    try:
        wb = load_workbook(data_dir,data_only=True)
        ws = wb[sheetName]
    except:
        with open('error.txt','w') as f:
            f.write('Cannot open the file\n')
            sys.exit('Cannot open the file')
    #clear former error.txt
    with open("error.txt", 'w') as file:
        file.truncate(0)
    inputLog, itemWeightNum, truckCapacity, cantLoadCargo = loadLoop(ws)
    writeInFile(inputLog, cantLoadCargo, truckCapacity)

In [11]:
try:
    main()
except:
    print('Error in running. Please check the error.txt')

Please input your data path/Users/jinchuchu/Documents/python/final/project.xlsx
Please input your sheetnameDATA
